# Prueba Tecnica

    Para poder realizar cada uno de los numerales deberas buscar una relacion entre las 2 tablas para consolidarlas en una sola. Posterior a esto podras empezar con la solucion de los puntos

    

    Nota:  
    - El punto A y B los debes solucionar tanto en Sql como python. El punto C lo podras realizar con el lenguaje de tu preferencia
    - para obtener las bases de datos solo deberas ejecutar el bloque de codigo llamado 'DATA' en caso de un error puedes pedir ayuda
    - Al terminar la prueba debes crear un repo en GitHub y nos debes mandar el link del repo al correo para revisarla (debe de ir todo el desarrollo y las bases para poder ejecutarla)

# Data

In [1]:
import pandas as pd
from os import path

'''
    Barrios: Información de los barrios (nombre y código)
    clientes: Información transaccional del cliente
    dispositivos: La descripcion y ubicacion de los dispositivos 
    
'''

df_barrios = pd.read_csv(path.join('Bases_insumo','barrios.csv'))
df_clientes = pd.read_csv(path.join('Bases_insumo','clientes.csv'))
df_dispositivos = pd.read_csv(path.join('Bases_insumo','dispositivos.csv'))
df_dispositivos.rename({'codigo':'cod_dispositivo'}, axis = 1, inplace= True)

df_join = df_barrios.merge(df_dispositivos, left_on ='codigo', 
                          right_on = 'id_barrio', how = 'inner').merge(df_clientes, on = 'cod_dispositivo', how = 'inner')


In [2]:
df_join.head(5)

,codigo,nombre,tipo,cod_dispositivo,latitud,longitud,id_barrio,num_doc,tipo_doc,canal,num_trx,mnt_total_trx
0,737998257,La Campina,POS,12202503,3.479572,-76.525582,737998257,-2393994463924208659,1,POS,2,12202503.0
1,737998257,La Campina,POS,10521045,3.480517,-76.525275,737998257,-5675531380015678594,1,POS,1,5260522.5
2,737998257,La Campina,POS,10521045,3.480517,-76.525275,737998257,-5066077049535851626,1,POS,1,5260522.5
3,737998257,La Campina,POS,13238282,3.479491,-76.525818,737998257,-5791329717693361241,1,POS,1,6619141.0
4,737998257,La Campina,POS,12567517,3.480958,-76.525153,737998257,-397966594878551344,1,POS,1,6283758.5


## Eliminemos columnas duplicadas

Vemos que codigo y id barrio estan duplicadas por tanto procedemos a depurar

In [3]:
variables_eliminar = ['id_barrio','canal']


In [4]:
df_join.drop(variables_eliminar, axis=1, inplace=True)
df_join.head()

,codigo,nombre,tipo,cod_dispositivo,latitud,longitud,num_doc,tipo_doc,num_trx,mnt_total_trx
0,737998257,La Campina,POS,12202503,3.479572,-76.525582,-2393994463924208659,1,2,12202503.0
1,737998257,La Campina,POS,10521045,3.480517,-76.525275,-5675531380015678594,1,1,5260522.5
2,737998257,La Campina,POS,10521045,3.480517,-76.525275,-5066077049535851626,1,1,5260522.5
3,737998257,La Campina,POS,13238282,3.479491,-76.525818,-5791329717693361241,1,1,6619141.0
4,737998257,La Campina,POS,12567517,3.480958,-76.525153,-397966594878551344,1,1,6283758.5


# Punto a

* Generar el listado de clientes con transacciones en 5 o más barrios.

### Codigo en SQL

In [5]:
from pandasql import sqldf

In [6]:
pysqldf = lambda q: sqldf(q, globals())

Select num_doc , count(distinct(nombre))
from df_join
group by num_doc

In [7]:
q = """SELECT 
            num_doc,
            count(distinct(nombre)) as Numero_de_barrios
       FROM df_join 
       GROUP BY num_doc;"""

resultado = pysqldf(q)

In [8]:
q = """SELECT 
            *
       FROM resultado 
       WHERE Numero_de_barrios>=5;"""

final = pysqldf(q)

In [9]:
final.to_excel('Bases_insumo\Respuestas\Rpunto a sql.xlsx')

In [10]:
q = """SELECT 
        count(distinct(num_doc))
       FROM final 
       """

conteo_sql = pysqldf(q)
conteo_sql

,count(distinct(num_doc))
0,3474


### Codigo en python

In [11]:
conte = df_join.groupby('num_doc')['nombre'].nunique()
conteo = pd.DataFrame(conte) 
conteo.head(5)

,nombre
num_doc,
-9218698523982046880,8
-9214605575224426554,1
-9211522780831135515,1
-9210956249921547631,1
-9210147727599394517,7


In [12]:
conteo.reset_index(inplace=True)
conteo.rename({'nombre':'Numero de barrios'},axis = 1, inplace=True)
conteo.head()

,num_doc,Numero de barrios
0,-9218698523982046880,8
1,-9214605575224426554,1
2,-9211522780831135515,1
3,-9210956249921547631,1
4,-9210147727599394517,7


In [13]:
conteo_filtro = conteo[conteo['Numero de barrios']>=5]

In [14]:
conteo_filtro.to_excel('Bases_insumo\Respuestas\Rpunto a Python.xlsx')

In [15]:
print('El numero total de clientes unicos es: ' + str(conteo.shape[0]))
print('El numero total de clientes unicos que han transado en cinco o mas barrios: ' + str(conteo_filtro.shape[0]))


El numero total de clientes unicos es: 10000
El numero total de clientes unicos que han transado en cinco o mas barrios: 3474


# Punto b

* ¿Cuáles son los 6 barrios donde la mayor cantidad de clientes únicos realizan transacciones en
dispositivos tipo PAC? La respuesta debe incluir la cantidad de clientes únicos asociados a estos
barrios.


### Codigo en SQL

In [16]:
r = """SELECT 
           nombre,
           count(distinct(num_doc)) as numero_clientes
       FROM df_join 
       WHERE tipo= "PAC"
       GROUP BY nombre
       order by 2 desc;"""

resultado_b = pysqldf(r)

In [17]:
resultado_b

,nombre,numero_clientes
0,San Nicolás,5
1,San Pedro,4
2,Colseguros Andes,4
3,Prados del Norte,1
4,Las Delicias,1
5,El Troncal,1


In [18]:
resultado_b.to_excel('Bases_insumo\Respuestas\Rpunto b SQL.xlsx',index = False)

### Codigo en python

In [19]:
df_join_filter = df_join[df_join['tipo']=="PAC"]
conteb = df_join_filter.groupby(['nombre'])['num_doc'].nunique().sort_values(ascending=False)
conteob = pd.DataFrame(conteb) 
conteob.reset_index(inplace=True)
conteob.head(5)

,nombre,num_doc
0,San Nicolás,5
1,Colseguros Andes,4
2,San Pedro,4
3,El Troncal,1
4,Las Delicias,1


In [20]:
conteob.to_excel('Bases_insumo\Respuestas\Rpunto b Python.xlsx',index = False)

In [21]:
conteob.head()

,nombre,num_doc
0,San Nicolás,5
1,Colseguros Andes,4
2,San Pedro,4
3,El Troncal,1
4,Las Delicias,1


Los barrios donde los clientes han realizado transacciones en canal PAC son: San Niocolás, San Pedro, Colseguros, Prados del Norte, Las Delicias y El Troncal. Son los únicos 6 barrios donde los clientes han realizado transacciones por este canal.

# Punto c

* Identifique cual es el mejor  y peor canal para cada uno de los barrios. Debes dar una explicacion teorica de como mejorarias el movimiento del mejor canal sin afectar al canal de estrella de cada barrio 


In [22]:
df_join.head()

,codigo,nombre,tipo,cod_dispositivo,latitud,longitud,num_doc,tipo_doc,num_trx,mnt_total_trx
0,737998257,La Campina,POS,12202503,3.479572,-76.525582,-2393994463924208659,1,2,12202503.0
1,737998257,La Campina,POS,10521045,3.480517,-76.525275,-5675531380015678594,1,1,5260522.5
2,737998257,La Campina,POS,10521045,3.480517,-76.525275,-5066077049535851626,1,1,5260522.5
3,737998257,La Campina,POS,13238282,3.479491,-76.525818,-5791329717693361241,1,1,6619141.0
4,737998257,La Campina,POS,12567517,3.480958,-76.525153,-397966594878551344,1,1,6283758.5


In [23]:
df_agg = df_join.groupby(['codigo','nombre','tipo']).agg({'num_trx': 'sum'}).sort_values(['codigo','num_trx'], ascending=False)

In [24]:
df_agg.reset_index(inplace=True)

In [25]:
a = """SELECT 
           codigo,
           nombre,
           tipo,
           num_trx,
           RANK() OVER (PARTITION BY codigo
                    ORDER BY num_trx asc
                    ) AS rank
       FROM df_agg ;"""

resultado_c = pysqldf(a)

In [26]:
resultado_c

,codigo,nombre,tipo,num_trx,rank
0,737995769,Julio Rincón,CB,52,1
1,737995769,Julio Rincón,POS,425,2
2,737995769,Julio Rincón,DISPENSADOR,3041,3
3,737995777,Los Rubles,POS,5,1
4,737996696,Aguacatal,CB,20,1
...,...,...,...,...,...
564,738018187,Urbanización Río Lili,DISPENSADOR,1696,5
565,738068452,Junin,CB,124,1
566,738068452,Junin,POS,128,2
567,738068452,Junin,SAI,627,3


In [27]:
b = """SELECT 
           codigo,
           max(rank) as canal_e,
           min(rank) as canal_o
       FROM resultado_c
        GROUP BY 1 ;"""

resultado_c2 = pysqldf(b)

In [28]:
resultado_c = resultado_c.merge(resultado_c2)

c = """
    SELECT 
           codigo,
           nombre,
           case when canal_e=rank then tipo else null end as estrella
       FROM resultado_c;"""

canal_e = pysqldf(c)



c = """
    SELECT *
       FROM canal_e
       WHERE estrella is not null;"""

canal_e = pysqldf(c)


In [29]:
canal_e

,codigo,nombre,estrella
0,737995769,Julio Rincón,DISPENSADOR
1,737995777,Los Rubles,POS
2,737996696,Aguacatal,CB
3,737996724,Terrón Colorado,CB
4,737996726,El Vergel,CB
...,...,...,...
275,738017956,Brisas del Limonar,POS
276,738017969,Las Acacias,DISPENSADOR
277,738018074,San Cristobal,DISPENSADOR
278,738018187,Urbanización Río Lili,DISPENSADOR


In [30]:
d = """
    SELECT 
           codigo,
           nombre,
           case when canal_o=rank then tipo else null end as oportunidad
       FROM resultado_c;"""

canal_o = pysqldf(d)

d = """
    SELECT *
       FROM canal_o
       WHERE oportunidad is not null;"""

canal_o = pysqldf(d)


In [31]:
df_c = df_join['codigo'].unique()
df_c = pd.DataFrame(df_c, columns=['codigo'])
df_c = df_c.merge(canal_e, how='left').merge(canal_o[['codigo','oportunidad']], how='left')


In [32]:


e = """
    SELECT codigo,
            nombre,
            estrella,
            oportunidad,
            case when estrella = oportunidad then 'SI' else 'NO' end as canal_unico
       FROM df_c;"""

df_c = pysqldf(e)

In [33]:
df_c

,codigo,nombre,estrella,oportunidad,canal_unico
0,737998257,La Campina,POS,POS,SI
1,737999435,Guayaquil,DISPENSADOR,CB,NO
2,737998219,Vipasa,DISPENSADOR,CB,NO
3,737996837,Lourdes,POS,POS,SI
4,737997031,Villanueva,POS,POS,SI
...,...,...,...,...,...
282,737997007,Alfonso López II,POS,POS,SI
283,737999402,Alfonso López III,POS,CB,NO
284,737998799,Ignacio Rengifo,POS,POS,SI
285,738000408,Prados del Limonar,DISPENSADOR,POS,NO


In [34]:
df_c.to_excel('Bases_insumo\Respuestas\Rpunto c.xlsx',index = False)

In [35]:
df_c['canal_unico'].value_counts()

NO    176
SI    111
Name: canal_unico, dtype: int64

El canal estrella y el canal oprtunidad (menos transacciones) para cada barrio puede visualizarse en la tabla df_c. El criterio utilizado para la selección de cada uno fue el número de trx. 

 Además.se ha creado una columna llamada 'canal_unico', como indicadora para aquellos barrios en los que el canal estrella es igual al canal oportunidad, es decir, tienen un solo canal de transacción. 

### Estrategias

De 288 barrios con transacciones, 176 solo tienen asociado un canal de transacción mientras que los 111 restantes si presentan un canal estrella y un canal oportunidad. Respecto a los primeros, la estrategia propuesta sería potenciar un segundo canal. La selección de éste sería a través de la identificación del canal estrella de los barrios cercanos a su zona.

Deben enfocarse los esfuerzos en canales más costoeficientes. Si el canal estrella es menos costoeficiente que el canal oportunidad, debería haber una especialización por el oportunidad, así ello implique que el estrella pierda participación. Sin embargo, si sucede lo contrario y el canal estrella es más costoeficiente, debería recomponerse la participación de los canales e incluso lograr mayor incidencia del canal estrella dentro del comportamiento transanccional de los barrios. 

### Canal Estrella por Barrio


A continuación, se puede observar gráficamente cómo es la distrbución de los barrios de Cali según su canal estrella. El canal estrella más representativo es el canal POS, seguido de Dispensador. 

<img src='Mapas\Canal estrella.png' alt="Alternative text" />

### Canal Oprtunidad por Barrio


Fundamentalmente en el centro de Cali, el canal POS es el único canal (estrella y oportunidad). A su alrededor, el canaal estrella es Dispensador por lo que debe ser aquel en el que más se profundice en esta zona.  

<img src='Mapas\Canal Oportunidad.png' alt="Alternative text" />

# Punto d

Realizar analisis exploratorio de los datos, sacar conclusiones y plantear acciones de mejora para el negocio

NOTA: Se debe agregar la documentación y trazabilidad del proceso realizado (Codigos y gráficas con las que se hicieron los análisis)

In [36]:
df_agg_d = df_join.groupby(['codigo']).agg({'num_trx': ['sum', 'mean'],
                                    'mnt_total_trx': ['sum', 'mean'],
                                    'num_doc' : pd.Series.nunique },
                                    group_keys=False)


In [37]:
df_agg_d

num_trx           mnt_total_trx               num_doc
              sum      mean           sum          mean nunique
codigo                                                         
737995769    3518  3.799136  4.491310e+10  4.850227e+07     357
737995777       5  1.000000  3.303009e+07  6.606018e+06       5
737996696      20  4.000000  2.047027e+10  4.094054e+09       5
737996724       3  3.000000  3.046240e+09  3.046240e+09       1
737996726       2  2.000000  1.823323e+09  1.823323e+09       1
...           ...       ...           ...           ...     ...
738017956       2  1.000000  1.479098e+07  7.395488e+06       2
738017969    1011  4.746479  2.499208e+10  1.173337e+08     162
738018074    1184  4.571429  1.508196e+10  5.823152e+07     187
738018187    3313  3.570043  1.529236e+10  1.647883e+07     551
738068452    1831  3.428839  1.203650e+11  2.254025e+08     429

[277 rows x 5 columns]

In [38]:
df_join.head()

,codigo,nombre,tipo,cod_dispositivo,latitud,longitud,num_doc,tipo_doc,num_trx,mnt_total_trx
0,737998257,La Campina,POS,12202503,3.479572,-76.525582,-2393994463924208659,1,2,12202503.0
1,737998257,La Campina,POS,10521045,3.480517,-76.525275,-5675531380015678594,1,1,5260522.5
2,737998257,La Campina,POS,10521045,3.480517,-76.525275,-5066077049535851626,1,1,5260522.5
3,737998257,La Campina,POS,13238282,3.479491,-76.525818,-5791329717693361241,1,1,6619141.0
4,737998257,La Campina,POS,12567517,3.480958,-76.525153,-397966594878551344,1,1,6283758.5


In [39]:
print('El numero de clientes unicos es: ' + str(df_join['num_doc'].unique().shape[0]))
print('El numero de transacciones: ' + str(df_join['num_trx'].sum()))
print('Monto total transado: ' + str(df_join['mnt_total_trx'].sum()/1000000000000) + ' billones')


El numero de clientes unicos es: 10000
El numero de transacciones: 304530
Monto total transado: 15.5360316679185 billones


### Top5 de Canales

In [40]:
df_join['tipo'].value_counts()

DISPENSADOR    37690
POS            37067
SAI             9374
CB              7076
MF              3386
PAC               17
Name: tipo, dtype: int64

### Top5 de Barrios: Participación Porcentual por TRX

In [41]:
df_join['nombre'].value_counts().head(5).sort_values(ascending=False)/df_join['num_trx'].sum()*100

San Pedro                   1.741700
Urbanización San Juaquín    1.429087
Santa Mónica Residencial    1.251437
Chipichape                  1.128624
Unicentro Cali              1.036679
Name: nombre, dtype: float64

Cabe resaltar que según el número de transacciones, se evidencia una transaccionalidad dispersa en toda la zona geográfica de Cali. Sin embargo, al revisar la concentración de clientes, vemos una participación relaiva más alta en el sur de la ciudad. 

<img src='Mapas\Clientes Unicos.png' alt="Alternative text" />

### Estadística descriptiva de TRX y Monto

In [42]:
df_join['num_trx'].describe()

count    94610.000000
mean         3.218793
std          6.272047
min          1.000000
25%          1.000000
50%          1.000000
75%          3.000000
max        315.000000
Name: num_trx, dtype: float64

In [43]:
df_join['mnt_total_trx'].describe()/1000000

count         0.094610
mean        164.211306
std        1357.421055
min           0.000030
25%           0.002569
50%           0.020430
75%           7.370069
max      123209.539564
Name: mnt_total_trx, dtype: float64

Respecto al número de transacciones y al monto transado, se puede evidenciar una alta desviación, indicando la presencia de valores atípicos sobre todo en los percentiles más altos. 

A continuación se puede evidenciar dódne se encuentra el nivel de transacciones per cápita más elevado (color naranja), incluso salta una alerta sobre las zonas rojas en donde las TRX per cápita superan el nivel de 10.

<img src='Mapas\trx percapita.png' alt="Alternative text" />

Respecto al monto percápita, deben evaluarse las zonas en rojo, en las que este valor supera el nivel de $1000 millones. 

<img src='Mapas\monto percapita.png' alt="Alternative text" />

### Conclusiones Generales

Existe una fuerte presencia de Nequi en todo el territorio caleño, sin embargo, para una gran porcentaje de los barrios, existe un único canal transaccional, predominantemente POS. Se sugiere enfocar los esfuerzos en una mayor penetración de segundos canales en estas zonas. 

Se evidencian algunas zonas de la ciduad con un nivel transaccional per cápita alto, en términos de número de transacciones per cápita y monto per cápita tranzado. Se sugiere revisar allí variables adicionales relacionadas con riesgo SARLAFT. 

Existe cerca de un 30% que ha transado en más de 5 barrios, evidenciando un porcentaje no despreciabel de clientes con una alta movilidad en la ciudad. Adicional, el comportamiento transacconal sobre todo en número de TRX está muy disperso por todo el territorio y no existe una alta concentración por barrios. Esta tendencia debería ser sostenible en el tiempo ya que coincide con una distribución homogénea de la presencia de Nequi en la ciudad. 





In [44]:
df_agg_d.to_excel('Bases_insumo\Respuestas\Barrios agregados.xlsx')